## SQLAlchemy

In [ ]:
#pip install sqlalchemy
#pip install psycopg2

In [1]:
username = "postgres"
password = "mohan"
host = "localhost"
port = "5432"
database_name = "python_learning"

In [2]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker

# Create an engine connected to the MySQL database
url = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database_name}"
engine = create_engine(url)

# Base class for our classes definitions
Base = declarative_base()

# Define a User model
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String(50))
    age = Column(Integer)

# Create the tables in the database
Base.metadata.create_all(engine)

In [3]:
# Create a configured "Session" class
Session = sessionmaker(bind=engine)
# Create a Session
session = Session()

### Create - INSERT

In [8]:
# Create a new user
new_user = User(name='John Doe', age=30)

# Add the new user to the session
session.add(new_user)

# Commit the session to write the user to the database
session.commit()

In [4]:
new_users = [User(name='Glenn Maxwell', age=34), User(name='Mike Stark', age=28), User(name='Paul Brandon', age=42)]
session.add_all(new_users)
session.commit()

### Read - SELECT

In [6]:
# Query all users
users = session.query(User).all()
for user in users:
    print(f"ID: {user.id}, Name: {user.name}, Age: {user.age}")

ID: 1, Name: John Doe, Age: 30
ID: 2, Name: Glenn Maxwell, Age: 34
ID: 3, Name: Mike Stark, Age: 28
ID: 4, Name: Paul Brandon, Age: 42


In [8]:
# Query a user by name
user = session.query(User).filter_by(name='John Doe').first()
print(f"ID: {user.id}, Name: {user.name}, Age: {user.age}")

ID: 1, Name: John Doe, Age: 30


In [11]:
#Query by age
users = session.query(User).filter_by(age = 30).all()
for user in users:
    print(f"ID: {user.id}, Name: {user.name}, Age: {user.age}")

ID: 1, Name: John Doe, Age: 30


### Update

In [13]:
# Query the user we want to update
user = session.query(User).filter_by(id = 3).first()

# Update the user's age
user.name = "Mitchell Stark"
user.age = 31

# Commit the session to write the changes to the database
session.commit()

### Delete

In [14]:
# Query the user we want to delete
user = session.query(User).filter_by(id = 4).first()

# Delete the user
session.delete(user)

# Commit the session to delete the user from the database
session.commit()

### Multiple Tables

In [84]:
from sqlalchemy import create_engine, Integer, String, DateTime, ForeignKey, Boolean
from sqlalchemy.orm import declarative_base, relationship, mapped_column, Mapped, sessionmaker
from typing import List
from time import localtime, strftime

# Create an engine connected to the MySQL database
url = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database_name}"
engine = create_engine(url)

# Base class for our classes definitions
Base = declarative_base()

In [85]:
class Customer(Base):
    __tablename__ = "customer"
    custId = mapped_column(name="cust_id", type_=Integer, primary_key=True, autoincrement="auto")
    custName = mapped_column(name="cust_name", type_=String(50), nullable=False)
    custPhone = mapped_column(name="cust_phone", type_=String(20))
    orders: Mapped[List["Order"]] = relationship(back_populates="customer")

class Order(Base):
    __tablename__ = "order"
    orderId = mapped_column("id", Integer, primary_key = True, autoincrement = "auto")
    orderDate = mapped_column("date", DateTime, nullable = False)
    orderItem = mapped_column("item", String)
    orderStatus = mapped_column("status", Boolean, default = True)
    custId: Mapped[int] = mapped_column("cust_id", ForeignKey("customer.cust_id"), nullable=False)
    createdAt = mapped_column("created_at", DateTime, default = strftime("%Y-%m-%d", localtime()))
    updatedAt = mapped_column("updated_at", DateTime)
    customer: Mapped["Customer"] = relationship(back_populates="orders")

# Create the tables in the database
Base.metadata.create_all(engine)